In [1]:
import os
import numpy as np
import scipy.io
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from kymatio.numpy import Scattering1D
from scipy import signal
from itertools import product
import time

In [2]:
# --- Paths ---
data_dir = 'training2017/training2017'
label_df = pd.read_csv("training2017/training2017/REFERENCE.csv", header=None, names=['filename', 'label'])
label_map = dict(zip(label_df['filename'], label_df['label']))

In [3]:
# --- Settings ---
fs = 300
samples_to_remove = 600
standard_ecg_length = 7800

In [4]:
# --- Preprocessing ---
def bandpass_filter(data, lowcut=0.05, highcut=100, fs=300, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data)


In [5]:
# Load and preprocess all ECG signals once
print("Loading and preprocessing ECG signals...")
preprocessed_signals = []
labels = []
file_list = sorted([f for f in os.listdir(data_dir) if f.endswith('.mat')])

for file in file_list:
    try:
        mat_path = os.path.join(data_dir, file)
        mat_contents = scipy.io.loadmat(mat_path)
        ecg = mat_contents['val'].squeeze()
        
        # Apply bandpass filter
        filtered = bandpass_filter(ecg)
        
        # Trim edges
        trimmed = filtered[samples_to_remove:-samples_to_remove]
        
        # Z-score normalize the signal
        trimmed = (trimmed - np.mean(trimmed)) / (np.std(trimmed) + 1e-10)
        
        # Standardize length
        if len(trimmed) < standard_ecg_length:
            standardized_ecg = np.pad(trimmed, (0, standard_ecg_length - len(trimmed)), mode='constant')
        else:
            start_idx = (len(trimmed) - standard_ecg_length) // 2
            standardized_ecg = trimmed[start_idx:start_idx + standard_ecg_length]
        
        preprocessed_signals.append(standardized_ecg)
        labels.append(label_map[file.replace('.mat', '')])
    except Exception as e:
        print(f"Error processing {file}: {e}")

Loading and preprocessing ECG signals...


In [6]:
# Convert labels to numeric
le = LabelEncoder()
y = le.fit_transform(labels)
print(f"Loaded {len(preprocessed_signals)} ECG signals")
print(f"Number of classes: {len(le.classes_)}")
# print(f"Class distribution: {np.bincount(y)}")

Loaded 8528 ECG signals
Number of classes: 4


In [7]:
# --- Train Final Model with Best Parameters ---
print("\n=== Training Final Model with Best Parameters ===")

# Extract features using best parameters
final_features = []
for signal in preprocessed_signals:
    scattering = Scattering1D(J=8, Q=6, shape=(standard_ecg_length,))
    coeffs = scattering(np.array(signal, dtype=np.float32))
    coeffs = coeffs.reshape(coeffs.shape[0], -1).flatten()
    final_features.append(coeffs)
X_final = np.array(final_features)



=== Training Final Model with Best Parameters ===


In [11]:
# # Run final evaluation with 10-fold CV
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_f1_scores = []
final_accuracies = []

# for fold, (train_idx, test_idx) in enumerate(skf.split(X_final, y), 1):
#     X_train, X_test = X_final[train_idx], X_final[test_idx]
#     y_train, y_test = y[train_idx], y[test_idx]
    
#     model = LGBMClassifier(n_estimators=100, learning_rate=0.05, max_depth=7, random_state=fold)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
    
#     acc = accuracy_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred, average='weighted')
#     final_accuracies.append(acc)
#     final_f1_scores.append(f1)
    
#     print(f"\n--- Final Model: Fold {fold} ---")
#     print(f"Accuracy: {acc:.4f}")
#     print(f"F1 Score (weighted): {f1:.4f}")
    
#     # if fold == 1:  # Print detailed report for first fold only
#     #     print(classification_report(y_test, y_pred, target_names=le.classes_))
        
#     #     # Confusion Matrix for first fold
#     #     plt.figure(figsize=(8, 6))
#     #     cm = confusion_matrix(y_test, y_pred)
#     #     sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_, cmap='Blues')
#     #     plt.title(f'Confusion Matrix - Best Model (J={J_best}, Q={Q_best})')
#     #     plt.xlabel("Predicted")
#     #     plt.ylabel("True")
#     #     plt.tight_layout()
#     #     plt.savefig('best_model_confusion_matrix.png')
#     #     plt.show()

# print("\n=== Final Results with Best Parameters ===")
# print(f"Best Parameters: J={8}, Q={6}")
# print(f"Average Accuracy: {np.mean(final_accuracies):.4f} ± {np.std(final_accuracies):.4f}")
# print(f"Average F1 Score: {np.mean(final_f1_scores):.4f} ± {np.std(final_f1_scores):.4f}")

In [23]:
#LightGBM
from sklearn.utils.class_weight import compute_class_weight

per_class_f1_scores = {label: [] for label in le.classes_}

for fold, (train_idx, test_idx) in enumerate(skf.split(X_final, y), 1):
    X_train, X_test = X_final[train_idx], X_final[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weights_dict = {i: w for i, w in enumerate(weights)}
    
    model = LGBMClassifier(n_estimators=497, learning_rate=0.05, max_depth=7, random_state=fold, class_weight=class_weights_dict)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    f1_per_class = f1_score(y_test, y_pred, average=None)

    final_accuracies.append(acc)
    final_f1_scores.append(f1_weighted)

    # Append per-class F1 scores
    for i, label in enumerate(le.classes_):
        per_class_f1_scores[label].append(f1_per_class[i])

    print(f"\n--- Final Model: Fold {fold} ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score (weighted): {f1_weighted:.4f}")
    for i, label in enumerate(le.classes_):
        print(f"F1 Score ({label}): {f1_per_class[i]:.4f}")

print("\n=== Final Results with Best Parameters ===")
print(f"Best Parameters: J=8, Q=6")
print(f"Average Accuracy: {np.mean(final_accuracies):.4f} ± {np.std(final_accuracies):.4f}")
print(f"Average F1 Score (weighted): {np.mean(final_f1_scores):.4f} ± {np.std(final_f1_scores):.4f}")

for label in le.classes_:
    mean_f1 = np.mean(per_class_f1_scores[label])
    std_f1 = np.std(per_class_f1_scores[label])
    print(f"Average F1 Score ({label}): {mean_f1:.4f} ± {std_f1:.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.133437 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1391280
[LightGBM] [Info] Number of data points in the train set: 7675, number of used features: 5456
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

C:\Users\sagar\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



--- Final Model: Fold 1 ---
Accuracy: 0.7808
F1 Score (weighted): 0.7710
F1 Score (A): 0.6767
F1 Score (N): 0.8651
F1 Score (O): 0.6233
F1 Score (~): 0.6222
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.157901 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1391280
[LightGBM] [Info] Number of data points in the train set: 7675, number of used features: 5456
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

C:\Users\sagar\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



--- Final Model: Fold 2 ---
Accuracy: 0.7761
F1 Score (weighted): 0.7635
F1 Score (A): 0.6667
F1 Score (N): 0.8616
F1 Score (O): 0.6175
F1 Score (~): 0.5333
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.111847 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1391280
[LightGBM] [Info] Number of data points in the train set: 7675, number of used features: 5456
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

...


--- Final Model: Fold 8 ---
Accuracy: 0.7808
F1 Score (weighted): 0.7724
F1 Score (A): 0.6565
F1 Score (N): 0.8700
F1 Score (O): 0.6316
F1 Score (~): 0.5600
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.401855 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1391280
[LightGBM] [Info] Number of data points in the train set: 7676, number of used features: 5456
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

C:\Users\sagar\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



--- Final Model: Fold 9 ---
Accuracy: 0.7782
F1 Score (weighted): 0.7699
F1 Score (A): 0.6822
F1 Score (N): 0.8595
F1 Score (O): 0.6297
F1 Score (~): 0.6122
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.208685 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1391280
[LightGBM] [Info] Number of data points in the train set: 7676, number of used features: 5456
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

C:\Users\sagar\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



--- Final Model: Fold 10 ---
Accuracy: 0.7653
F1 Score (weighted): 0.7580
F1 Score (A): 0.5574
F1 Score (N): 0.8637
F1 Score (O): 0.6324
F1 Score (~): 0.4783

=== Final Results with Best Parameters ===
Best Parameters: J=8, Q=6
Average Accuracy: 0.7607 ± 0.0192
Average F1 Score (weighted): 0.7565 ± 0.0158
Average F1 Score (A): 0.6424 ± 0.0355
Average F1 Score (N): 0.8648 ± 0.0084
Average F1 Score (O): 0.6301 ± 0.0122
Average F1 Score (~): 0.5654 ± 0.0656


In [24]:
import pandas as pd
df = pd.DataFrame(per_class_f1_scores)
df.columns = ['Class A', 'Class N', 'Class O', 'Class ~']
df.index = [f'Fold {i+1}' for i in range(10)]
df['F1 with noise'] = df.mean(axis=1)
df['F1 without noise'] = df[['Class A', 'Class N', 'Class O']].mean(axis=1)

average_f1_with_noise = df['F1 with noise'].mean()
average_f1_without_noise = df['F1 without noise'].mean()

variance_f1_with_noise = df['F1 with noise'].var()
variance_f1_without_noise = df['F1 without noise'].var()

print(df)

          Class A   Class N   Class O   Class ~  F1 with noise  \
Fold 1   0.676692  0.865065  0.623318  0.622222       0.696824   
Fold 2   0.666667  0.861566  0.617512  0.533333       0.669770   
Fold 3   0.656000  0.862419  0.617117  0.666667       0.700551   
Fold 4   0.602941  0.872558  0.624719  0.600000       0.675055   
Fold 5   0.634921  0.847970  0.619958  0.600000       0.675712   
Fold 6   0.650000  0.862963  0.649123  0.440000       0.650521   
Fold 7   0.641221  0.881801  0.655098  0.541667       0.679947   
Fold 8   0.656489  0.869972  0.631579  0.560000       0.679510   
Fold 9   0.682171  0.859535  0.629712  0.612245       0.695916   
Fold 10  0.557377  0.863680  0.632444  0.478261       0.632940   

         F1 without noise  
Fold 1           0.721692  
Fold 2           0.715248  
Fold 3           0.711845  
Fold 4           0.700073  
Fold 5           0.700949  
Fold 6           0.720695  
Fold 7           0.726040  
Fold 8           0.719346  
Fold 9           0.72

In [25]:
for label in le.classes_:
    mean_f1 = np.mean(per_class_f1_scores[label])
    std_f1 = np.std(per_class_f1_scores[label])
    print(f"Average F1 Score ({label}): {mean_f1:.4f} ± {std_f1:.4f}")

print(f"\nAverage F1 with noise: {average_f1_with_noise:.4f} ± {variance_f1_with_noise:.4f}")
print(f"Average F1 without noise: {average_f1_without_noise:.4f} ± {variance_f1_without_noise:.4f}")

Average F1 Score (A): 0.6424 ± 0.0355
Average F1 Score (N): 0.8648 ± 0.0084
Average F1 Score (O): 0.6301 ± 0.0122
Average F1 Score (~): 0.5654 ± 0.0656

Average F1 with noise: 0.6757 ± 0.0004
Average F1 without noise: 0.7124 ± 0.0002
